In [1]:
from janome.tokenizer import Tokenizer

t = Tokenizer()

text = '東京都でおいしいビールを飲もう'
tokens = t.tokenize(text)

#文章を分割する
for token in tokens:
    print(token)

東京	名詞,固有名詞,地域,一般,*,*,東京,トウキョウ,トーキョー
都	名詞,接尾,地域,*,*,*,都,ト,ト
で	助詞,格助詞,一般,*,*,*,で,デ,デ
おいしい	形容詞,自立,*,*,形容詞・イ段,基本形,おいしい,オイシイ,オイシイ
ビール	名詞,一般,*,*,*,*,ビール,ビール,ビール
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
飲も	動詞,自立,*,*,五段・マ行,未然ウ接続,飲む,ノモ,ノモ
う	助動詞,*,*,*,不変化型,基本形,う,ウ,ウ


In [2]:
#varsはオブジェクトの属性を教えてくれる
vars(tokens[0])

{'surface': '東京',
 'part_of_speech': '名詞,固有名詞,地域,一般',
 'infl_type': '*',
 'infl_form': '*',
 'base_form': '東京',
 'reading': 'トウキョウ',
 'phonetic': 'トーキョー',
 'node_type': 'SYS_DICT'}

In [3]:
text2 = 'うらにわには　にわにわにわ、にわとりがいる。'
#ひらがなでは解析できていない。
tokens2 = t.tokenize(text2)
for token in tokens2:
    print(token)

うら	動詞,自立,*,*,五段・ラ行,未然形,うる,ウラ,ウラ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
わに	名詞,一般,*,*,*,*,わに,ワニ,ワニ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
　	記号,空白,*,*,*,*,　,　,
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
わに	名詞,一般,*,*,*,*,わに,ワニ,ワニ
わに	名詞,一般,*,*,*,*,わに,ワニ,ワニ
わ	助詞,終助詞,*,*,*,*,わ,ワ,ワ
、	記号,読点,*,*,*,*,、,、,、
にわとり	名詞,一般,*,*,*,*,にわとり,ニワトリ,ニワトリ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
いる	動詞,自立,*,*,一段,基本形,いる,イル,イル
。	記号,句点,*,*,*,*,。,。,。


In [4]:
text3 = '裏庭には二羽、庭には二羽、鶏がいる。'
tokens3 = t.tokenize(text3, wakati=True)
for token in tokens3:
    print(token)

裏庭
に
は
二
羽
、
庭
に
は
二
羽
、
鶏
が
いる
。


In [5]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
sentences = ['おいしいビールを飲む', 'コーヒーを飲む', 'おいしいクラフトビールを買う']

words_list =[]
for sentence in sentences:
    words_list.append(t.tokenize(sentence, wakati=True))
words_list

[['おいしい', 'ビール', 'を', '飲む'],
 ['コーヒー', 'を', '飲む'],
 ['おいしい', 'クラフト', 'ビール', 'を', '買う']]

In [6]:
#重複の無い単語リスト
unique_words = []
for words in words_list:
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
unique_words

['おいしい', 'ビール', 'を', '飲む', 'コーヒー', 'クラフト', '買う']

In [7]:
#Bag of Wordsのデータを作成する
bow_list =[]
for words in words_list:
    bag_of_words = [] #文章のBag of Words　を格納する
    for unique_word in unique_words:
        num = words.count(unique_word) #一意な単語の出現回数を数える
        bag_of_words.append(num)
    bow_list.append(bag_of_words)
bow_list

[[1, 1, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0], [1, 1, 1, 0, 0, 1, 1]]

In [12]:
unique_words

['おいしい', 'ビール', 'を', '飲む', 'コーヒー', 'クラフト', '買う']

In [8]:
#pandasを用いて見やすく表示
import pandas as pd
df = pd.DataFrame(bow_list, columns = unique_words, index=sentences)
df

,おいしい,ビール,を,飲む,コーヒー,クラフト,買う
おいしいビールを飲む,1,1,1,1,0,0,0
コーヒーを飲む,0,0,1,1,1,0,0
おいしいクラフトビールを買う,1,1,1,0,0,1,1


In [18]:
#IDFを計算する
from math import log #logをインポート
num_of_sentences = len(sentences)
idf = []
for i in range(len(unique_words)): #一意な単語分繰り返す
    count = 0
    for bow in bow_list:
        if bow[i] > 0:
            count += 1
    idf.append(log((num_of_sentences + 1) / (count + 1))) #単語のIDFを計算
        
df = pd.Series(idf, index =unique_words)
df

おいしい    0.287682
ビール     0.287682
を       0.000000
飲む      0.287682
コーヒー    0.693147
クラフト    0.693147
買う      0.693147
dtype: float64

３つすべての文章で出現する単語「を」の
IDFは0になる。IDFが小さいほど、どの
文章にも現れる単語であることを示す

In [25]:
#TFを計算する
tf_list = []
for bow in bow_list: 
    num_of_words = sum(bow)　
    tf = []
    for n in bow:
        tf.append(n / num_of_words)
    tf_list.append(tf)

import pandas as pd
df = pd.DataFrame(tf_list, columns=unique_words, index=sentences)
df

,おいしい,ビール,を,飲む,コーヒー,クラフト,買う
おいしいビールを飲む,0.25,0.25,0.250000,0.250000,0.000000,0.0,0.0
コーヒーを飲む,0.00,0.00,0.333333,0.333333,0.333333,0.0,0.0
おいしいクラフトビールを買う,0.20,0.20,0.200000,0.000000,0.000000,0.2,0.2


In [28]:
import numpy as np
idf_a = np.array(idf)
tf_list_a = np.array(idf)
tf_list_a = np.array(tf_list)
tfidf_list = tf_list_a *(idf_a + 1)

df = pd.DataFrame(tfidf_list, columns=unique_words, index=sentences)
df

,おいしい,ビール,を,飲む,コーヒー,クラフト,買う
おいしいビールを飲む,0.321921,0.321921,0.250000,0.321921,0.000000,0.000000,0.000000
コーヒーを飲む,0.000000,0.000000,0.333333,0.429227,0.564382,0.000000,0.000000
おいしいクラフトビールを買う,0.257536,0.257536,0.200000,0.000000,0.000000,0.338629,0.338629
